# Playing around with the Spotify API

In [1]:
import pandas as pd
import numpy as np
import os
import sys
import spotipy
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy.util as util

from scipy import spatial

Using TensorFlow backend.


In [2]:

os.environ['SPOTIPY_CLIENT_ID'] = "cba9273cc2784d68b10cab9a7f7a25e6" 
os.environ['SPOTIPY_CLIENT_SECRET'] = "5df1e92959334eaca52ccdcb279f31c6" 
os.environ['SPOTIPY_REDIRECT_URI'] = "http://localhost:8888/callback" 

#export SPOTIPY_CLIENT_ID='cba9273cc2784d68b10cab9a7f7a25e6'
#!export SPOTIPY_CLIENT_SECRE='5df1e92959334eaca52ccdcb279f31c6'T
#!export SPOTIPY_REDIRECT_URI='http://localhost:8888/callback'

In [5]:
'''
define the API connection - usually log in from here as well...
'''
token = util.prompt_for_user_token('', '')

client_credentials_manager = SpotifyClientCredentials()
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)


#spotify=spotipy.Spotify()




            User authentication requires interaction with your
            web browser. Once you enter your credentials and
            give authorization, you will be redirected to
            a url.  Paste that url you were directed to to
            complete the authorization.

        
Opened https://accounts.spotify.com/authorize?client_id=cba9273cc2784d68b10cab9a7f7a25e6&redirect_uri=http%3A%2F%2Flocalhost%3A8888%2Fcallback&response_type=code in your browser


Enter the URL you were redirected to: http://localhost:8888/callback?code=AQB4IpJENLHZ0UFizPlyBmG9pn3i5snN8C3BujCqtdvA_RGOg-xufE6UKz2He1ybSdVPmQUhDIoav8E6Xpj45WXDOQU_Yt6UEgMiRfWl8MbaYWQiDRvzvFwDiMo0KtSuvQwdHxdeM3N8w9_f7hfvmp_3oVCNnXE22u8qbW5gMqA4PtB9zUTcQAAYxE4zvLOmmEE90w




In [6]:
'''example with finding a users albums... '''
birdy_uri = 'spotify:artist:2WX2uTcsvV5OnS0inACecP' #try and make a function
#that returns this 'uri' given artist name.

results = sp.artist_top_tracks(birdy_uri, country = 'SE')
#albums = results['items']
print(results)

#while results['next']:
#    results = spotify.next(results)
#    albums.extend(results['items'])

#for album in albums:
#    print(album['name'])

{'tracks': [{'popularity': 78, 'artists': [{'uri': 'spotify:artist:2WX2uTcsvV5OnS0inACecP', 'href': 'https://api.spotify.com/v1/artists/2WX2uTcsvV5OnS0inACecP', 'type': 'artist', 'external_urls': {'spotify': 'https://open.spotify.com/artist/2WX2uTcsvV5OnS0inACecP'}, 'name': 'Birdy', 'id': '2WX2uTcsvV5OnS0inACecP'}], 'external_ids': {'isrc': 'GBAHS1100350'}, 'duration_ms': 201080, 'href': 'https://api.spotify.com/v1/tracks/4RL77hMWUq35NYnPLXBpih', 'album': {'id': '1WGjSVIw0TVfbp5KrOFiP0', 'artists': [{'uri': 'spotify:artist:2WX2uTcsvV5OnS0inACecP', 'href': 'https://api.spotify.com/v1/artists/2WX2uTcsvV5OnS0inACecP', 'type': 'artist', 'external_urls': {'spotify': 'https://open.spotify.com/artist/2WX2uTcsvV5OnS0inACecP'}, 'name': 'Birdy', 'id': '2WX2uTcsvV5OnS0inACecP'}], 'available_markets': ['AR', 'AT', 'BE', 'BG', 'BO', 'BR', 'CH', 'CL', 'CO', 'CR', 'CZ', 'DE', 'DK', 'DO', 'EC', 'EE', 'FI', 'GR', 'GT', 'HK', 'HN', 'HU', 'ID', 'IE', 'IS', 'IT', 'JP', 'LT', 'LU', 'LV', 'MX', 'MY', 'NI', 

In [7]:
''' 
luke i changed ur function down below the index 1 was wrong so i put it as  0 and it giving
way better results, chur...
'''
def get_uri(artist):
    results = sp.search(q = "artist:" + artist, type = "artist")
    artist_id = results['artists']['items'][0]['uri'] # changed from '1' to '0' ??
   
    return(artist_id)

get_uri("Beyonce")
#A function that returns artist 'uri' as a string. 
#By most matching (first result)

'spotify:artist:6vWDO969PvNqNYHIOW5v0m'

In [54]:
sp.categories()

{'categories': {'href': 'https://api.spotify.com/v1/browse/categories?offset=0&limit=20',
  'items': [{'href': 'https://api.spotify.com/v1/browse/categories/toplists',
    'icons': [{'height': 275,
      'url': 'https://t.scdn.co/media/derived/toplists_11160599e6a04ac5d6f2757f5511778f_0_0_275_275.jpg',
      'width': 275}],
    'id': 'toplists',
    'name': 'Top Lists'},
   {'href': 'https://api.spotify.com/v1/browse/categories/mood',
    'icons': [{'height': 274,
      'url': 'https://t.scdn.co/media/original/mood-274x274_976986a31ac8c49794cbdc7246fd5ad7_274x274.jpg',
      'width': 274}],
    'id': 'mood',
    'name': 'Mood'},
   {'href': 'https://api.spotify.com/v1/browse/categories/party',
    'icons': [{'height': 274,
      'url': 'https://t.scdn.co/media/links/partyicon_274x274.jpg',
      'width': 274}],
    'id': 'party',
    'name': 'Party'},
   {'href': 'https://api.spotify.com/v1/browse/categories/pop',
    'icons': [{'height': 274,
      'url': 'https://t.scdn.co/media/deri

In [8]:
''' lets try get related song '''

artst = 'Young Thug'

# get audio analysis of SP track skinny love...
related = sp.audio_features('spotify:track:4RL77hMWUq35NYnPLXBpih')

print(related)
#for artist in related['artists']:
    #print('  ', artist['name'])
    #print('  ', artist['popularity'])
    #print('  ', artist['followers']['total'])



[{'analysis_url': 'https://api.spotify.com/v1/audio-analysis/4RL77hMWUq35NYnPLXBpih', 'danceability': 0.379, 'acousticness': 0.952, 'uri': 'spotify:track:4RL77hMWUq35NYnPLXBpih', 'key': 4, 'loudness': -8.485, 'duration_ms': 201080, 'tempo': 166.467, 'speechiness': 0.051, 'instrumentalness': 0.00106, 'energy': 0.29, 'valence': 0.176, 'mode': 1, 'time_signature': 4, 'type': 'audio_features', 'liveness': 0.118, 'id': '4RL77hMWUq35NYnPLXBpih', 'track_href': 'https://api.spotify.com/v1/tracks/4RL77hMWUq35NYnPLXBpih'}]


In [9]:
def related_artists(name):
    related = sp.artist_related_artists(get_uri(name))
    rels = []
    for artist in related['artists']:
        rels.append(artist['name'])
    return(rels)


def next_artist(artist, artists = []):
    ''' artists is the storage of past people that have been in there'''
    RelatedArtists = related_artists(artist)
    relates = [] # empty list to append with future 
    
    for i in RelatedArtists:
        relates.append(related_artists(i) )
    
    # create a bag of words - to index each artist in:
    
    BOW = np.unique(relates).flatten() 
    vecs = np.zeros((len(relates), len(BOW)))  # unique artists as columns...

    for i in range(len(relates)):
        # turn each into a vector of 0 and 1
        for art in relates[i]:
            # this loops over each artist:
            idx = np.where(BOW == art)[0][0]
            # set that idx number = to 1
            vecs[i,idx] = 1
    
    
    # then calculate cosine similarity and select the lowest similarity artist and print them
    sims = np.zeros((len(relates),len(relates)))
    sim_means = []
    for i in range(len(vecs)):
        # loop over each vector
        for j in range(len(vecs)):
            # do this again because you want to get similarity between each matrix...
            sims[i,j] = 1 - spatial.distance.cosine(vecs[i],vecs[j])
        # calculate row wise means:
        sim_means.append(np.var(sims[i] ))
    
    # find the lowest scoring similarity artist
    lowest_idx = np.where(sim_means == max(sim_means) )[0][0]
    #print(lowest_idx)
    # return artists name:
    

    #if (RelatedArtists[lowest_idx] in artists) == True :
    # do the process again...
    
    
    print(RelatedArtists[lowest_idx])
    
    
    return(RelatedArtists[lowest_idx])

In [44]:
next_artist('Firehouse')

White Lion


'White Lion'

In [12]:
# import some data to autoencode -- top artists
os.chdir('C:\\Users\\Max.Polaczuk\\Google Drive\\music_merge\\artists')
pop50 = pd.read_csv('pop50.csv')

In [16]:
print(pop50.ix[:,0].tolist())
pop = (pop50.ix[:,0].tolist() )

['THE BEATLES', 'MADONNA', 'ELTON JOHN', 'ELVIS PRESLEY', 'MARIAH CAREY', 'STEVIE WONDER', 'JANET JACKSON', 'MICHAEL JACKSON', 'WHITNEY HOUSTON', 'THE ROLLING STONES', 'PAUL MCCARTNEY', 'BEE GEES', 'RIHANNA', 'USHER', 'CHICAGO', 'THE SUPREMES', 'DARYL HALL JOHN OATES', 'PRINCE', 'ROD STEWART', 'OLIVIA NEWTON-JOHN', 'ARETHA FRANKLIN', 'MARVIN GAYE', 'PHIL COLLINS', 'KATY PERRY', 'BILLY JOEL', 'DIANA ROSS', 'THE 4 SEASONS', 'THE TEMPTATIONS', 'DONNA SUMMER', 'THE BEACH BOYS', 'LIONEL RICHIE', 'NEIL DIAMOND', 'CARPENTERS', 'TAYLOR SWIFT', 'BOYZ II MEN', 'THE JACKSONS', 'CONNIE FRANCIS', 'KENNY ROGERS', 'BEYONCE', 'BRENDA LEE', 'BARBRA STREISAND', 'BRYAN ADAMS', 'CHER', 'MAROON 5', 'GEORGE MICHAEL', 'THE BLACK EYED PEAS', 'BOBBY VINTON', 'JOHN MELLENCAMP', 'THREE DOG NIGHT', 'HUEY LEWIS & THE NEWS']


In [17]:
## for each artist get their uid and save

uris = []

for p in pop:
    uris.append(get_uri(p))
    
print(uris)

['spotify:artist:3WrFJ7ztbogyGnTHbHJFl2', 'spotify:artist:6tbjWDEIzxoDsBA1FuhfPW', 'spotify:artist:3PhoLpVuITZKcymswpck5b', 'spotify:artist:43ZHCT0cAZBISjO8DG9PnE', 'spotify:artist:4iHNK0tOyZPYnBU7nGAgpQ', 'spotify:artist:7guDJrEfX3qb6FEbdPA5qi', 'spotify:artist:4qwGe91Bz9K2T8jXTZ815W', 'spotify:artist:3fMbdgg4jU18AjLCKBhRSm', 'spotify:artist:6XpaIBNiVzIetEPCWDvAFP', 'spotify:artist:22bE4uQ6baNwSHPVcDxLCe', 'spotify:artist:4STHEaNw4mPZ2tzheohgXB', 'spotify:artist:1LZEQNv7sE11VDY3SdxQeN', 'spotify:artist:5pKCCKE2ajJHZ9KAiaK11H', 'spotify:artist:23zg3TcAtWQy7J6upgbUnj', 'spotify:artist:07d5etnpjriczFBB8pxmRe', 'spotify:artist:57bUPid8xztkieZfS7OlEV', 'spotify:artist:77tT1kLj6mCWtFNqiOmP9H', 'spotify:artist:3MHaV05u0io8fQbZ2XPtlC', 'spotify:artist:2y8Jo9CKhJvtfeKOsYzRdT', 'spotify:artist:4BoRxUdrcgbbq1rxJvvhg9', 'spotify:artist:7nwUJBm0HE4ZxD3f5cy5ok', 'spotify:artist:3koiLjNrgRTNbOwViDipeA', 'spotify:artist:4lxfqrEsLX6N1N4OCSkILp', 'spotify:artist:6jJ0s89eD6GaHleKKya26X', 'spotify:artist

In [22]:
## now we can use these fellows to return their artist information:

# save each genre in a list 
genres = []
for i in range(len(uris)):
    K = sp.artist(uris[i])['genres']
    for k in range(len(K)):
        genres.append(K[k])

print(genres)

['british invasion', 'classic rock', 'merseybeat', 'protopunk', 'psychedelic rock', 'rock', 'dance pop', 'europop', 'new wave pop', 'pop', 'pop rock', 'album rock', 'classic rock', 'folk christmas', 'glam rock', 'mellow gold', 'piano rock', 'rock', 'singer-songwriter', 'soft rock', 'christmas', 'rock', 'rock-and-roll', 'rockabilly', 'dance pop', 'pop', 'pop christmas', 'post-teen pop', 'r&b', 'soul christmas', 'urban contemporary', 'adult standards', 'funk rock', 'motown', 'soft rock', 'soul', 'soul christmas', 'dance pop', 'hip pop', 'indie r&b', 'motown', 'neo soul', 'new jack swing', 'pop', 'pop rap', 'pop rock', 'quiet storm', 'r&b', 'soul', 'urban contemporary', 'dance pop', 'pop', 'pop christmas', 'dance pop', 'europop', 'hip pop', 'pop', 'pop christmas', 'quiet storm', 'r&b', 'soul', 'soul christmas', 'urban contemporary', 'album rock', 'british invasion', 'classic rock', 'folk rock', 'mellow gold', 'protopunk', 'rock', 'singer-songwriter', 'album rock', 'classic rock', 'folk ro

In [25]:
# get the unique list of genres and index them..
genres = np.unique(genres)
print(genres)

['adult standards' 'album rock' 'australian dance' 'bachata' 'blues-rock'
 'boy band' 'brill building pop' 'british blues' 'british invasion'
 'bubblegum pop' 'cabaret' 'canadian pop' 'chicago soul' 'christmas'
 'classic funk rock' 'classic rock' 'country' 'country christmas'
 'country rock' 'dance pop' 'dance rock' 'deep indie r&b' 'deep pop r&b'
 'disco' 'doo-wop' 'easy listening' 'electric blues' 'escape room'
 'europop' 'folk' 'folk christmas' 'folk rock' 'funk' 'funk rock'
 'glam rock' 'hard rock' 'hip hop' 'hip pop' 'hollywood' 'indie r&b'
 'jazz blues' 'latin' 'latin hip hop' 'latin pop' 'lounge' 'mellow gold'
 'memphis soul' 'merengue urbano' 'merseybeat' 'motown' 'nashville sound'
 'neo mellow' 'neo soul' 'new jack swing' 'new romantic' 'new wave'
 'new wave pop' 'opera' 'operatic pop' 'outlaw country' 'piano rock' 'pop'
 'pop christmas' 'pop rap' 'pop rock' 'post-teen pop' 'protopunk'
 'psychedelic rock' 'quiet storm' 'r&b' 'rap' 'reggaeton' 'rock'
 'rock-and-roll' 'rockabill

In [ ]:
# setup null matrix length of artists by length of genres...
GEN = np.zeros((len(pop),len(genres)))

# run a loop over each artist to get their vector of 

for i in range(len(uris)):
    K = sp.artist(uris[i])['genres']
    for k in range(len(K)):
        idx = np.where(genres == K[k])[0][0]
        GEN[i,idx] = 1

In [30]:
print(GEN)

91